In [2]:
import ee
ee.Authenticate()

Enter verification code:  4/1AdQt8qgxWReYu_Ff8hYGKgGqXVnVJ1J9Npn3h9XpGn8WO1Z07Do4MMVSZw0



Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap
import glob
import boto3

In [5]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Read input data

In [22]:
# define directory
out_dir = os.getcwd()
aws_s3_dir = "https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data"

In [23]:
# read Tree Mosaic Land
tml = ee.ImageCollection('projects/wri-datalab/TML')
tml_tree_cover = tml.reduce(ee.Reducer.mean()).rename('b1')

In [37]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')

# remove cities without tree cover data availability
tml_not_available_cities = ['BRA-Salvador','MEX-Monterrey']
boundary_georef = boundary_georef[~boundary_georef['geo_name'].isin(tml_not_available_cities)].reset_index(drop=True)
boundary_georef

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM3,COD,ADM3
1,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
2,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
3,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
4,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
5,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
6,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
7,Antananarivo,MDG-Antananarivo,ADM4union,ADM4,MDG-Antananarivo-ADM4,MDG,ADM4
8,Mexico_City,MEX-Mexico_City,ADM2union,ADM2,MEX-Mexico_City-ADM2,MEX,ADM2
9,Musanze,RWA-Musanze,ADM5union,ADM5,RWA-Musanze-ADM5,RWA,ADM5


# Compute indicator

In [38]:
cities_indicators_sicb_11 = pd.DataFrame() 

In [39]:
for i in range(0, len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
    
    # process aoi level ------
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # compute average tml
    tml_mean = tml_tree_cover.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': boundary_geo_ee.geometry(),
        'scale': 10,
        'maxPixels': 1e9
    })
    # get output
    tml_mean_dict = tml_mean.getInfo()
    tml_mean_value = tml_mean_dict['b1']
    
    # store in df and apend
    df = geemap.ee_to_pandas(boundary_geo_ee)
    df['percent_of_tree_cover'] = tml_mean_value
    cities_indicators_sicb_11 = cities_indicators_sicb_11.append(df)
    
    # process unit of analysis level ------
    print("\n boundary_id_unit: "+boundary_id_unit)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
     # compute average tml
    tml_mean = tml_tree_cover.reduceRegions(**{
        'reducer': ee.Reducer.mean(),
        'collection': boundary_geo_ee,
        'scale': 10
    })
    
    # store output in df and apend
    df = geemap.ee_to_pandas(tml_mean)
    df = df.rename(columns={"mean": "percent_of_tree_cover"})
    cities_indicators_sicb_11 = cities_indicators_sicb_11.append(df)


0

 geo_name: COD-Bukavu

 boundary_id_aoi: COD-Bukavu-ADM3union

 boundary_id_unit: COD-Bukavu-ADM3
1

 geo_name: COD-Uvira

 boundary_id_aoi: COD-Uvira-ADM3union

 boundary_id_unit: COD-Uvira-ADM3
2

 geo_name: COG-Brazzaville

 boundary_id_aoi: COG-Brazzaville-ADM4union

 boundary_id_unit: COG-Brazzaville-ADM4
3

 geo_name: COL-Barranquilla

 boundary_id_aoi: COL-Barranquilla-ADM4union

 boundary_id_unit: COL-Barranquilla-ADM4
4

 geo_name: ETH-Addis_Ababa

 boundary_id_aoi: ETH-Addis_Ababa-ADM4union

 boundary_id_unit: ETH-Addis_Ababa-ADM4
5

 geo_name: ETH-Dire_Dawa

 boundary_id_aoi: ETH-Dire_Dawa-ADM3union

 boundary_id_unit: ETH-Dire_Dawa-ADM3
6

 geo_name: KEN-Nairobi

 boundary_id_aoi: KEN-Nairobi-ADM3union

 boundary_id_unit: KEN-Nairobi-ADM3
7

 geo_name: MDG-Antananarivo

 boundary_id_aoi: MDG-Antananarivo-ADM4union

 boundary_id_unit: MDG-Antananarivo-ADM4
8

 geo_name: MEX-Mexico_City

 boundary_id_aoi: MEX-Mexico_City-ADM2union

 boundary_id_unit: MEX-Mexico_City-ADM2
9

In [40]:
cities_indicators_sicb_11

,geo_parent_name,geo_level,creation_date,geo_id,geo_name,percent_of_tree_cover
0,COD-Bukavu,ADM3-union,2022-08-03,COD-Bukavu_ADM3-union_1,COD-Bukavu,13.268002
0,COD-Bukavu,ADM3,2022-08-03,COD-Bukavu_ADM3_1,Kadutu,15.544192
1,COD-Bukavu,ADM3,2022-08-03,COD-Bukavu_ADM3_2,Ibanda,13.512293
2,COD-Bukavu,ADM3,2022-08-03,COD-Bukavu_ADM3_3,Bagira,12.047980
0,COD-Uvira,ADM3-union,2022-08-03,COD-Uvira_ADM3-union_1,COD-Uvira,26.105939
...,...,...,...,...,...,...
135,RWA-Musanze,ADM5,2022-08-26,RWA-Musanze_ADM5_136,Burengo,28.526825
136,RWA-Musanze,ADM5,2022-08-26,RWA-Musanze_ADM5_137,Bwamazi,NaN
137,RWA-Musanze,ADM5,2022-08-26,RWA-Musanze_ADM5_138,Kadahenda,NaN
138,RWA-Musanze,ADM5,2022-08-26,RWA-Musanze_ADM5_139,Karwesero,NaN


# Merge with indicator table

In [30]:
# read indicator table
cities_indicators = pd.read_csv(aws_s3_dir + '/indicators/cities_indicators_v2.csv') 
cities_indicators

,geo_id,geo_level,geo_name,geo_parent_name
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador
1,BRA-Salvador_ADM4_1,ADM4,Pituaçu,BRA-Salvador
2,BRA-Salvador_ADM4_2,ADM4,Patamares,BRA-Salvador
3,BRA-Salvador_ADM4_3,ADM4,Piatã,BRA-Salvador
4,BRA-Salvador_ADM4_4,ADM4,Boca do Rio,BRA-Salvador
...,...,...,...,...
1077,RWA-Musanze_ADM5_136,ADM5,Burengo,RWA-Musanze
1078,RWA-Musanze_ADM5_137,ADM5,Bwamazi,RWA-Musanze
1079,RWA-Musanze_ADM5_138,ADM5,Kadahenda,RWA-Musanze
1080,RWA-Musanze_ADM5_139,ADM5,Karwesero,RWA-Musanze


In [31]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace by new indicators")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    else:
        print("add new indicators")
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    return(cities_indicators_df)

In [32]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = cities_indicators_sicb_11,
                                            indicator_name = "percent_of_tree_cover")

add new indicators


In [33]:
cities_indicators_merged

,geo_id,geo_level,geo_name,geo_parent_name,percent_of_tree_cover
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,NaN
1,BRA-Salvador_ADM4_1,ADM4,Pituaçu,BRA-Salvador,NaN
2,BRA-Salvador_ADM4_2,ADM4,Patamares,BRA-Salvador,NaN
3,BRA-Salvador_ADM4_3,ADM4,Piatã,BRA-Salvador,NaN
4,BRA-Salvador_ADM4_4,ADM4,Boca do Rio,BRA-Salvador,NaN
...,...,...,...,...,...
1077,RWA-Musanze_ADM5_136,ADM5,Burengo,RWA-Musanze,28.526825
1078,RWA-Musanze_ADM5_137,ADM5,Bwamazi,RWA-Musanze,NaN
1079,RWA-Musanze_ADM5_138,ADM5,Kadahenda,RWA-Musanze,NaN
1080,RWA-Musanze_ADM5_139,ADM5,Karwesero,RWA-Musanze,NaN


# Upload in aws s3

In [34]:
# connect to s3
aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [35]:
# upload to aws
key_data = 'data/indicators/dev/cities_indicators_v2.csv'
bucket_name = 'cities-cities4forests' 
cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [36]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')